In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import spatial_maps as sp
import septum_mec.analysis.data_processing as dp
import septum_mec.analysis.registration
import expipe
import os
import pathlib
import scipy
import scipy.signal
import numpy as np
import exdir
import pandas as pd
import optogenetics as og
import quantities as pq
import shutil
from distutils.dir_util import copy_tree
import elephant as el
import neo
from scipy.signal import find_peaks
from scipy.interpolate import interp1d
from matplotlib import mlab

from spike_statistics.core import theta_mod_idx

from tqdm import tqdm_notebook as tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

16:20:19 [I] klustakwik KlustaKwik2 version 0.2.6
/home/mikkel/.virtualenvs/expipe/lib/python3.6/site-packages/ipykernel_launcher.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`


In [3]:
data_loader = dp.Data()
actions = data_loader.actions
project = data_loader.project

In [4]:
output = pathlib.Path('output/theta-rhythmicity')
(output / 'data').mkdir(parents=True, exist_ok=True)

In [5]:
identify_neurons = actions['identify-neurons']
# sessions = pd.read_csv(identify_neurons.data_path('sessions'))
units = pd.read_csv(identify_neurons.data_path('units'))

In [6]:
def get_lim(action_id):
    stim_times = data_loader.stim_times(action_id)
    if stim_times is None:
        return [0, np.inf]
    stim_times = np.array(stim_times)
    return [stim_times.min(), stim_times.max()]


def compute_stim_freq(action_id):
    stim_times = data_loader.stim_times(action_id)
    if stim_times is None:
        return np.nan
    stim_times = np.array(stim_times)
    return 1 / np.mean(np.diff(stim_times))

In [11]:
def signaltonoise(a, axis=0, ddof=0):
    a = np.asanyarray(a)
    m = a.mean(axis)
    sd = a.std(axis=axis, ddof=ddof)
    return np.where(sd == 0, 0, m / sd)

In [7]:
def compute_clean_lfp(anas, width=500, threshold=2):
    anas = np.array(anas)
    idxs, = np.where(abs(anas) > threshold)
    for idx in idxs:
        anas[idx-width:idx+width] = 0 # TODO AR model prediction
    return anas, idxs


def compute_clean_spikes(spikes, idxs, times, width=500):

    for idx in idxs:
        t0 = times[idx-width]
        stop = idx + width
        if stop > len(times) - 1:
            stop = len(times) - 1 
        t1 = times[stop]
        mask = (spikes > t0) & (spikes < t1)
        spikes = spikes[~mask]
    spikes = spikes[spikes <= times[-1]]
    return spikes


def prepare_spike_lfp(anas, sptr, t_start, t_stop):

    t_start = t_start * pq.s if t_start is not None else 0 * pq.s
    sampling_rate = anas.sampling_rate
    units = anas.units
    times = anas.times
    if t_start is not None and t_stop is not None:
        t_stop = t_stop * pq.s
        mask = (times > t_start) & (times < t_stop)
        anas = np.array(anas)[mask,:]
        times = times[mask]
    
    # take best channel from other drive
    best_channel = np.argmax(signaltonoise(anas))
#     best_channel = np.random.choice(anas.shape[1])
    
    cleaned_anas, idxs = compute_clean_lfp(anas[:, best_channel])
    
    cleaned_anas = neo.AnalogSignal(
        signal=cleaned_anas * units, sampling_rate=sampling_rate, t_start=t_start
    )
    
    spike_units = sptr.units
    spike_times = sptr.times
    spike_times = compute_clean_spikes(spike_times, idxs, times)

    sptr = neo.SpikeTrain(
        spike_times[(spike_times > t_start) & (spike_times < times[-1])], units=spike_units,
        t_start=t_start, t_stop=times[-1]
    )

    return cleaned_anas, sptr, best_channel

In [8]:
NFFT = 8192
theta_band_f1, theta_band_f2 = 6, 10 

In [17]:
coherence_data, freqency_data = {}, {}
theta_kde_data, theta_bins_data = {}, {}
stim_kde_data, stim_bins_data = {}, {}

def process(row):
    action_id = row['action']
    channel_group = row['channel_group']
    unit_name = row['unit_name']
    
    name = f'{action_id}_{channel_group}_{unit_name}'
    
    # select 
    lfp = data_loader.lfp(action_id, channel_group) # TODO consider choosing strongest stim response
    
    sptr = data_loader.spike_train(action_id, channel_group, unit_name)
    
    lim = get_lim(action_id)
    
    cleaned_lfp, sptr, best_channel = prepare_spike_lfp(lfp, sptr, *lim)
    
    if len(sptr) == 0:
        theta_score = np.nan
    else:
        theta_score = theta_mod_idx(np.array(sptr))
    
    result = pd.Series({
        'theta_score': theta_score,
    })
    return result

In [18]:
results = units.merge(
    units.progress_apply(process, axis=1), 
    left_index=True, right_index=True)

/home/mikkel/apps/expipe-project/spike-statistics/spike_statistics/core.py:27: RuntimeWarning: invalid value encountered in double_scalars
  return (pk - th)/(pk + th)


In [16]:
%debug

> <__array_function__ internals>(6)concatenate()



ipdb>  up


> /home/mikkel/.virtualenvs/expipe/lib/python3.6/site-packages/pyqtgraph/numpy_fix.py(13)concatenate()
     11         names = [getattr(dt, 'names', None) for dt in dtypes]
     12         if len(dtypes) < 2 or all([n is None for n in names]):
---> 13             return np.concatenate_orig(vals, *args, **kwds)
     14         if any([dt != dtypes[0] for dt in dtypes[1:]]):
     15             raise TypeError("Cannot concatenate structured arrays of different dtype.")



ipdb>  up


> /home/mikkel/apps/expipe-project/spike-statistics/spike_statistics/core.py(776)correlogram()
    774     # We have excluded spikes outside of the histogram range to limit
    775     # memory use here.
--> 776     big = np.concatenate([t2[i:j] - t for t, i, j in zip(t1, ii2, jj2)])
    777 
    778     # Actually do the histogram. Note that calls to np.histogram are



...

ipdb>  data_loader.spike_train(action_id, channel_group, unit_name)


<SpikeTrain(array([883.25376667, 883.25536667, 883.25573333, 884.45873333,
       884.459     , 884.4604    , 884.46096667]) * 1/Hz, [0.0 1/Hz, 956.1173 1/Hz])>


ipdb>  len(data_loader.spike_train(action_id, channel_group, unit_name))


7


ipdb>  exit


# Save to expipe

In [ ]:
action = project.require_action("theta-rhytmicity")

In [ ]:
action.data['results'] = 'results.csv'
results.to_csv(action.data_path('results'), index=False)

In [ ]:
copy_tree(output, str(action.data_path()))

In [ ]:
septum_mec.analysis.registration.store_notebook(action, "10-calculate-theta-rhythmicity.ipynb")